In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from sklearn.cross_validation import train_test_split
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras_self_attention import SeqSelfAttention
from keras.utils import np_utils
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers import Input


sess = tf.Session()
K.set_session(sess)

dfin = pd.read_csv('../NNNormalizeDataIn.csv',header=None)  #inputs 128 lines per sample
dfout = pd.read_csv('../NNNormalizeDataOutClassses.csv',header=None)  #output classes 

total_inputs,total_output = dfin.as_matrix().astype(np.float32),dfout.as_matrix().astype(np.int32)

total_inputs = np.reshape(total_inputs, (-1,128,9))

train_inputs, test_inputs, train_output, test_output = train_test_split(total_inputs, total_output, test_size=0.2, random_state=42)

NB_EPOCH = 5
# network and training
BATCH_SIZE = 64
VERBOSE = 1
OPTIMIZER = Adam()
VALIDATION_SPLIT=0.2

IMG_ROWS, IMG_COLS = 128, 5 
NB_CLASSES = 5 
INPUT_SHAPE = (1, IMG_ROWS, IMG_COLS)

X_train = train_inputs
y_train = np_utils.to_categorical(train_output, NB_CLASSES)
X_test = test_inputs
y_test = np_utils.to_categorical(test_output, NB_CLASSES)


class LSTMNet:
    @staticmethod
    def build(timeSteps,variables,classes):
        inputNet = Input(shape=(timeSteps,variables)) 
       
        lstm=Bidirectional(LSTM(100,recurrent_dropout=0.4,dropout=0.4,return_sequences=True),merge_mode='concat')(inputNet) #worse using stateful=True
        lstm=Bidirectional(LSTM(50,recurrent_dropout=0.4,dropout=0.4,return_sequences=False),merge_mode='concat')(lstm) #worse using stateful=True 
        
        classificationLayer=Dense(classes,activation='softmax')(lstm)
        
        model=Model(inputNet,classificationLayer)
        return model
    
modellstm = LSTMNet.build(timeSteps=128,variables=9,classes=5)
modellstm.summary()

#K.categorical_crossentropy

modellstm.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER,metrics=["accuracy"])


#modellstm.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER,metrics=["accuracy"])

sess.run(tf.global_variables_initializer())



Using TensorFlow backend.
/home/rjpg/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 9)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128, 200)          88000     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100)               100400    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 505       
Total params: 188,905
Trainable params: 188,905
Non-trainable params: 0
_________________________________________________________________


In [2]:


history = modellstm.fit(X_train, y_train, 
        batch_size=BATCH_SIZE, epochs=NB_EPOCH, 
        verbose=2,
        validation_data=(X_test,y_test))

Train on 1660 samples, validate on 415 samples
Epoch 1/5
46s - loss: 1.6078 - acc: 0.2169 - val_loss: 1.5977 - val_acc: 0.2434
Epoch 2/5
55s - loss: 1.5996 - acc: 0.2295 - val_loss: 1.5844 - val_acc: 0.2675
Epoch 3/5
56s - loss: 1.5830 - acc: 0.2620 - val_loss: 1.5759 - val_acc: 0.2578
Epoch 4/5
53s - loss: 1.5815 - acc: 0.2723 - val_loss: 1.5732 - val_acc: 0.2771
Epoch 5/5
50s - loss: 1.5694 - acc: 0.2783 - val_loss: 1.5669 - val_acc: 0.2723


In [5]:
modellstm2 = LSTMNet.build(timeSteps=128,variables=9,classes=5)
modellstm2.summary()

modellstm2.compile(loss=K.categorical_crossentropy, optimizer=OPTIMIZER,metrics=["accuracy"])

sess.run(tf.global_variables_initializer())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 9)            0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128, 200)          88000     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 100)               100400    
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 505       
Total params: 188,905
Trainable params: 188,905
Non-trainable params: 0
_________________________________________________________________


In [6]:


history = modellstm2.fit(X_train, y_train, 
        batch_size=BATCH_SIZE, epochs=NB_EPOCH, 
        verbose=2,
        validation_data=(X_test,y_test))

Train on 1660 samples, validate on 415 samples
Epoch 1/5
53s - loss: 12.8783 - acc: 0.2114 - val_loss: 12.8372 - val_acc: 0.2361
Epoch 2/5
59s - loss: 12.7791 - acc: 0.2446 - val_loss: 12.7565 - val_acc: 0.2193
Epoch 3/5
62s - loss: 12.6016 - acc: 0.2494 - val_loss: 12.6343 - val_acc: 0.2217
Epoch 4/5
57s - loss: 12.3212 - acc: 0.2452 - val_loss: 12.3410 - val_acc: 0.2410
Epoch 5/5
64s - loss: 12.0311 - acc: 0.2693 - val_loss: 12.2590 - val_acc: 0.2337
